In [69]:
import csv
from datasets import load_dataset

original_dataset = load_dataset("potsawee/wiki_bio_gpt3_hallucination")
original_dataset

DatasetDict({
    evaluation: Dataset({
        features: ['gpt3_text', 'wiki_bio_text', 'gpt3_sentences', 'annotation', 'wiki_bio_test_idx', 'gpt3_text_samples'],
        num_rows: 238
    })
})

In [70]:
ans = []
labels = []

# Iterate over each passage and its corresponding label
for i, (passage, label) in enumerate(zip(original_dataset['evaluation']['gpt3_sentences'], original_dataset['evaluation']['annotation'])):
    sentences = passage
    
    # Check for mismatch in the number of sentences and labels
    if len(sentences) != len(label):
        print(f"Mismatch at index {i}: Sentences - {len(sentences)}, Labels - {len(label)}")
    
    ans.extend(sentences)
    labels.extend(label)

In [71]:
print(len(ans))
print(len(labels))

1908
1908


In [72]:
data = list(zip(range(1, len(ans) + 1), ans, labels))
csv_file_path = "./data/generated/selfcheckgpt.csv"

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['qid', 'ans', 'label'])
    
    # Write the data
    writer.writerows(data)

print(f"CSV file has been created at {csv_file_path}")

CSV file has been created at ./data/generated/selfcheckgpt.csv
